In [ ]:
cache = {}
last_checkpoint_time = time.time()
MAX_CACHE_SIZE = 1000
CHECKPOINT_INTERVAL = 3600  
CACHE_CHECKPOINT_DIR = "cache_checkpoint"
os.makedirs(CACHE_CHECKPOINT_DIR, exist_ok=True)

mysql_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Smalik@72",
    database="final"
)
mysql_cursor = mysql_connection.cursor()

def generate_cache_key(query):
    return hashlib.md5(str(query).encode()).hexdigest()

def fetch_data_with_caching(query):
    cache_key = generate_cache_key(query)
    if cache_key in cache:
        print("Data fetched from cache")
        cache[cache_key]['access_time'] = time.time()
        return cache[cache_key]['data']
    
    data = None
    try:
        if isinstance(query, tuple):  
            sql_query, params = query
            print("Data fetched from MySQL")
            mysql_cursor.execute(sql_query, params)
            data = mysql_cursor.fetchall()  # This will fetch as a list of tuples
            columns = [desc[0] for desc in mysql_cursor.description]
            data = [dict(zip(columns, row)) for row in data]  # Convert to list of dicts
        elif isinstance(query, dict) and 'aggregate' in query:  # MongoDB aggregation (no equivalent for MySQL)
            print("Aggregation not supported in MySQL")
            return None
        elif isinstance(query, dict):  # Simple select operation
            print("Data fetched from MySQL using find")
            table_name = query['table']
            filter_clause = query.get('filter', '')
            projection_clause = query.get('projection', '*')
            sql_query = f"SELECT {projection_clause} FROM {table_name} WHERE {filter_clause}"
            mysql_cursor.execute(sql_query)
            data = mysql_cursor.fetchall()
            columns = [desc[0] for desc in mysql_cursor.description]
            data = [dict(zip(columns, row)) for row in data]  # Convert to list of dicts
    except Exception as e:
        print(f"An error occurred: {e}")
    
    if len(cache) >= MAX_CACHE_SIZE:
        evict_least_accessed()
    cache[cache_key] = {'data': data, 'access_time': time.time()}
    possibly_checkpoint_cache()
    return data

def evict_least_accessed():
    min_access_time = float('inf')
    min_cache_key = None
    for key, value in cache.items():
        if float(value['access_time']) < min_access_time:
            min_access_time = float(value['access_time'])
            min_cache_key = key
    if min_cache_key:
        del cache[min_cache_key]
        print("Evicted cache entry")

def possibly_checkpoint_cache():
    if time.time() - last_checkpoint_time >= CHECKPOINT_INTERVAL:
        checkpoint_cache()

def checkpoint_cache():
    global last_checkpoint_time
    checkpoint_file = os.path.join(CACHE_CHECKPOINT_DIR, "cache_checkpoint.json")
    with open(checkpoint_file, 'w') as f:
        json.dump(cache, f, default=str)
    print("Cache checkpointed to disk")
    last_checkpoint_time = time.time()

def load_cache_from_checkpoint():
    checkpoint_file = os.path.join(CACHE_CHECKPOINT_DIR, "cache_checkpoint.json")
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            loaded_cache = json.load(f)
        return loaded_cache
    return {}